# Voca Quiz Mid and High prompt

In [2]:
from pydantic import BaseModel
from openai import OpenAI
from openai.lib._parsing import type_to_response_format_param
import json
from getpass import getpass

openai_api_key = getpass("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

In [3]:
from jinja2 import Template

prompt_template = Template("""
[PURPOSE]
You are going to generate fill-in-the-blank quiz and its Korean translation for vocabularies based on the given input. The quizzes must ask the words themselves written in the example_sentence. You must provide adequate hints for each question with Korean translation.

[QUIZ GUIDELINES]
- You must create one quizzes per word.
- You must put blanks on the GIVEN word, NOWHERE ELSE.
- You must provide Korean translation.
- You must put blanks on the matching part of Korean translation.
- You must choose ONLY ONE type of hints.
- You must shuffle your hint type to distribute them evenly.

[HINT 1: ALPHABET]
- As a hint, provide the first letter of the word.
	e.g.) Input: "believe"
				Output: ["It starts with 'b'!", "b로 시작하는 단어예요!"]

[HINT 2: UNSCRAMBLE]
- As a hint, scramble the letters of the word.
	e.g.) Input: "flood"
				Output: ["Rearrange these letters!: L O F D O", "다음 글자들을 재배열 해보세요!: L O F D O"]
				
[HINT 3: LETTER]
- This only applies to a word that is under 8 letters, and non-phrases. As a hint, provide the number of the letters in the word.
	e.g.) Input: "example"
				Output: ["It has 7 letters!", "7글자예요!"]

[HINT 4: EXPLAIN]
- As a hint, explain the word in other ways.
	e.g.) Input 1: "glasses"
				Output 1: ["You wear 'these' to see better.", "더 잘 볼 수 있도록 '이것'을 써요."]
				Input 2: "excited"
				Output 2: ["How would you feel when you go on a field trip?", "수학여행을 갈 때 기분이 어떨 것 같아요?"]

[POLICY]
Each quiz needs:
- A blank to test the word. You must not include any grammatical suffix if any.
- A blank in the Korean translation as well.
- Shuffle the hint types to prevent getting boring.

[Example]
```
Input: 
    {
    "voca_NO": "1234"
    "WordText": "command",
    "WordMeaning": "명령하다",
    "PoS": "1"
    "PoSText": "동사"
  }

Output:
    {
    "voca_quizzes":
      {
        "fill_in_blank_quiz": "The officer ___ed the soldiers to move forward.",
        "korean_translation": "장교는 군인들에게 전진하라고 [[명령]]했어요.",
        "answer": "command",
      },
    "hint": 
	    {
		    "hint_eng": "Rearrange these letters!: O A N M M C D"
		    "hint_kor": "다음 글자들을 재배열 해보세요!: O A N M M C D"
		  }

}
```

[Input]
```
VOCA_NO : {{voca_NO}}
WORDTEXT : {{WordText}}
WORDMEANING : {{WordMeaning}}
POSTEXT : {{PoSText}}
```
""")

In [3]:
prompt = prompt_template.render(
    voca_NO = "104",
    WordText = "ordinary",
    WordMeaning = "평범한",
    PoSText = "형용사"
) # 샘플 넣는 곳

In [4]:
print(prompt)


[PURPOSE]
You are going to generate fill-in-the-blank quiz and its Korean translation for vocabularies based on the given input. The quizzes must ask the words themselves written in the example_sentence. You must provide adequate hints for each question with Korean translation.

[QUIZ GUIDELINES]
- You must create one quizzes per word.
- You must put blanks on the GIVEN word, NOWHERE ELSE.
- You must provide Korean translation.
- You must put blanks on the matching part of Korean translation.
- You must choose ONLY ONE type of hints.
- You must shuffle your hint type to distribute them evenly.

[HINT 1: ALPHABET]
- As a hint, provide the first letter of the word.
	e.g.) Input: "believe"
				Output: ["It starts with 'b'!", "b로 시작하는 단어예요!"]

[HINT 2: UNSCRAMBLE]
- As a hint, scramble the letters of the word.
	e.g.) Input: "flood"
				Output: ["Rearrange these letters!: L O F D O", "다음 글자들을 재배열 해보세요!: L O F D O"]
				
[HINT 3: LETTER]
- This only applies to a word that is under 8 letters

In [4]:
class VocaQuiz(BaseModel):
    fill_in_blank_quiz : str
    korean_translation : str
    answer : str

class QuizHint(BaseModel):
    hint_eng : str
    hint_kor : str

class QuizResponse(BaseModel):
    voca_quizzes : list[VocaQuiz]
    hint : list[QuizHint]

In [5]:
response_format = type_to_response_format_param(QuizResponse)

In [6]:
response_format

{'type': 'json_schema',
 'json_schema': {'schema': {'$defs': {'QuizHint': {'properties': {'hint_eng': {'title': 'Hint Eng',
       'type': 'string'},
      'hint_kor': {'title': 'Hint Kor', 'type': 'string'}},
     'required': ['hint_eng', 'hint_kor'],
     'title': 'QuizHint',
     'type': 'object',
     'additionalProperties': False},
    'VocaQuiz': {'properties': {'fill_in_blank_quiz': {'title': 'Fill In Blank Quiz',
       'type': 'string'},
      'korean_translation': {'title': 'Korean Translation', 'type': 'string'},
      'answer': {'title': 'Answer', 'type': 'string'}},
     'required': ['fill_in_blank_quiz', 'korean_translation', 'answer'],
     'title': 'VocaQuiz',
     'type': 'object',
     'additionalProperties': False}},
   'properties': {'voca_quizzes': {'items': {'$ref': '#/$defs/VocaQuiz'},
     'title': 'Voca Quizzes',
     'type': 'array'},
    'hint': {'items': {'$ref': '#/$defs/QuizHint'},
     'title': 'Hint',
     'type': 'array'}},
   'required': ['voca_quizzes

In [8]:
def completion(prompt : str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini',
        reasoning_effort='low',
        messages = [
            {"role" : "system", "content" : "You will provide a fill-in-the-blank quiz for each given word, a Korean translation, and a hint."},
            {"role" : "user", "content" : prompt}
        ],
        response_format = QuizResponse,
    )
    return response.choices[0].message.parsed

In [9]:
response = completion(prompt)

In [10]:
response

QuizResponse(voca_quizzes=[VocaQuiz(fill_in_blank_quiz='The movie was so ______ that nothing about it was memorable.', korean_translation='그 영화는 너무 [[평범한]]해서 특별함을 느낄 수 없었어요.', answer='ordinary')], hint=[QuizHint(hint_eng='Rearrange these letters!: R D I O Y N A R', hint_kor='다음 글자들을 재배열 해보세요!: R D I O Y N A R')])

In [11]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent = 4)
print(response_output)

{
    "voca_quizzes": [
        {
            "fill_in_blank_quiz": "The movie was so ______ that nothing about it was memorable.",
            "korean_translation": "그 영화는 너무 [[평범한]]해서 특별함을 느낄 수 없었어요.",
            "answer": "ordinary"
        }
    ],
    "hint": [
        {
            "hint_eng": "Rearrange these letters!: R D I O Y N A R",
            "hint_kor": "다음 글자들을 재배열 해보세요!: R D I O Y N A R"
        }
    ]
}


# for문 돌려서 Batch API jsonl 파일 형식 만들기

확인 사항
- A1, A2, B1, B2, C1, C2 CEFR level 별로 퀴즈만 만들어지면 되는건가?
- 여섯개의 레벨 모두 만들어야하는지, 일부 레벨(A1, B1, C1)만 만들면 되는지도 결정

1. 커리큘럼 정보 csv 파일 읽어오기
2. for문으로 한줄씩 순회하면서 프롬프트 만들어주기들어주기
3. 렌더링한 프롬프트 파일에 써주기 (처음엔 조금만, ex. 5개 커리큘럼)
4. openai batch API 요청해서 task_id 받아오기 
5. batch task 완료되면 결과 받아오고 검수하기

In [ ]:
def VocaQuizMake(data, output_filename):
    jsonl_data = []

    for i in range(len(data)):
        prompt = prompt_template.render(
            voca_NO = data.loc[i,"voca NO"],
            WordText = data.loc[i,"WordText"],
            WordMeaning = data.loc[i,"WordMeaning"],
            PoSText = data.loc[i,"품사"]
        )

        quiz_request = {
            "custom_id" : f"request-{i+1}",
            "method" : "POST",
            "url" : "/v1/chat/completions",
            "body" : {
                "model" : "o3-mini-2025-01-31",
                "messages" : [
                    {"role": "system", "content": "You will provide a fill-in-the-blank quiz for each given word, a Korean translation, and a hint."},
                    {"role": "user", "content": prompt}
                ],
                "response_format" : response_format
            }
        }

        jsonl_data.append(quiz_request)

        with open(output_filename, 'w', encoding='utf-8') as jsonl_file:
            for item in jsonl_data:
                jsonl_file.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f'JSONL 파일 생성 완료 : {output_filename}-{i+1}')

## 중학교 jsonl 파일

In [13]:
import pandas as pd

raw_data = pd.read_csv("../ExampleSentence_Word/Mid/Voca_MidHigh.csv",skiprows=1)

display(raw_data.head(3))

,voca NO,grade ID,grade NO,WordText,WordMeaning,PoS,품사,image ID,WordID(SoundID),WordSymbol,example_english,example_korean
0,1,NaN,NaN,accent,강세,2,명사,5094,14274,[ǽksent],NaN,NaN
1,2,NaN,NaN,accent,억양,2,명사,5133,20699,[ǽksent],NaN,NaN
2,3,NaN,NaN,advertise,광고하다,1,동사,420,671,[ǽdvərtàiz],NaN,NaN


In [14]:
raw_data.columns

Index(['voca NO', 'grade ID', 'grade NO', 'WordText', 'WordMeaning', 'PoS',
       '품사', 'image ID', 'WordID(SoundID)', 'WordSymbol', 'example_english',
       'example_korean'],
      dtype='object')

In [15]:
VocaQuizMake(raw_data, output_filename = 'VocaQuiz_MidHigh_batch.jsonl')

JSONL 파일 생성 완료 : VocaQuiz_MidHigh_batch.jsonl-2842


In [13]:
import pandas as pd
df = pd.read_csv("../ExampleSentence_Word/Mid/Final_Voca_MidHigh.csv", skiprows=2)
sample = df[:10]
sample.head(3)

,구분,voca NO,grade ID,grade NO,WordText,WordMeaning,PoS,품사,image ID,WordID(SoundID),WordSymbol,example_english,example_korean,중고등1200 포함여부,Unnamed: 14,워드P 매핑용,워드P 매핑용 쿼리
0,중고등1200,1,NaN,NaN,accent,강세,2,명사,5094,14274,[ǽksent],Studying many foreign languages often enhances...,많은 외국어를 공부하면 발음이 향상되는데 특히 각 단어의 올바른 강세를 꼼꼼히 익히...,True,NaN,NaN,NaN
1,중고등1200,2,NaN,NaN,accent,억양,2,명사,5133,20699,[ǽksent],Her charming accent made her speech both capti...,그녀의 매력적인 억양은 모든 대화에서 그녀의 말을 매료되고 쉽게 이해할 수 있게 했어요.,True,NaN,NaN,NaN
2,중고등1200,3,중2,7.0,advertise,광고하다,1,동사,420,671,[ǽdvərtàiz],We need to advertise our new product effective...,우리는 최대한의 도달 효과를 위해 다양한 미디어 채널을 통해 새로운 제품을 광고해야...,False,NaN,NaN,NaN


In [22]:
def VocaQuizMake(data):
    for i in range(len(data)):
        prompt = prompt_template.render(
            voca_NO = data.loc[i,"voca NO"],
            WordText = data.loc[i,"WordText"],
            WordMeaning = data.loc[i,"WordMeaning"],
            PoSText = data.loc[i,"품사"]
        )
    
        response = client.beta.chat.completions.parse(
            model = 'o3-mini',
            reasoning_effort='low',
            messages = [
                {"role" : "system", "content" : "You will provide a fill-in-the-blank quiz for each given word, a Korean translation, and a hint."},
                {"role" : "user", "content" : prompt}
            ],
            response_format = QuizResponse,
        )
        response_output = response.choices[0].message.parsed
        final_output = json.dumps(response_output.dict(), ensure_ascii=False, indent = 4)
        print(f"{i+1} 번째 quiz : {final_output}")
        print("\n")

In [23]:
VocaQuizMake(sample)

1 번째 quiz : {
    "voca_quizzes": [
        {
            "fill_in_blank_quiz": "The speaker's unique ______ enriched his speech.",
            "korean_translation": "그 화자의 독특한 [[강세]]가 그의 말투를 풍부하게 만들었어요.",
            "answer": "accent"
        }
    ],
    "hint": [
        {
            "hint_eng": "It starts with 'a'!",
            "hint_kor": "a로 시작하는 단어예요!"
        }
    ]
}


2 번째 quiz : {
    "voca_quizzes": [
        {
            "fill_in_blank_quiz": "She speaks with a distinct ___.",
            "korean_translation": "그녀는 독특한 [[억양]]으로 말해요.",
            "answer": "accent"
        }
    ],
    "hint": [
        {
            "hint_eng": "It starts with 'a'!",
            "hint_kor": "a로 시작하는 단어예요!"
        }
    ]
}


3 번째 quiz : {
    "voca_quizzes": [
        {
            "fill_in_blank_quiz": "The company chose to ___ their new product during the gala event.",
            "korean_translation": "그 회사는 신제품을 [[광고]]하기로 결정했어요.",
            "answer": "advertise"
        }
    